<a href="https://colab.research.google.com/github/g4aidl-upc-spring-2021/3D-PointCloud-Classification/blob/master/Final_PointNet_document.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Check GPU
In order to perform the experiments in a reasonable time, check whether the GPU has at least 15GiB. If it is not, it's necessary to restart the runtime until this requirement is satisfied

In [ ]:
!nvidia-smi

Mon Jul 12 19:10:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Installations and imports


### Installations
As some libraries that are not in the default version in colab are used, it is necessary to install them

In [ ]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-geometric -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html

!pip install torchmetrics

Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
     |████████████████████████████████| 2.6MB 6.2MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
     |████████████████████████████████| 1.4MB 6.0MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
     |████████████████████████████████| 931kB 7.7MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
     |████████████████████████████████| 225kB 6.6MB/s 
     |████████████████████████████████| 235kB 10.4MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.7.2-cp37-none-any.whl size=388143 sha256=5565828c1d5d587ca28ed9953d6e582569b90d2112e35bb8b5b4a73bdf31c2d9
  Stored in directory: /root/.cache/pip/wheels/e7/66/5b/ad17ef7f04b7c425dc6930daac160c3747231b0d65f9ac7972
Successfully built torch-geometric
     |████████████████████████████████| 235kB 11.7MB/s 


### Imports
In the next snippet of code there are all the imports necessaries for the project and the tensorboard is initialized.

In [ ]:
import os
import datetime
import numpy as np

import tensorflow
import tensorboard
import plotly.graph_objects as go

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.tensorboard import SummaryWriter

from torchmetrics import Accuracy

from torch_geometric.data import DataLoader
from torch_geometric.utils import to_dense_batch
from torch_geometric.datasets import ModelNet
from torch_geometric.transforms import SamplePoints, NormalizeScale, RandomFlip, RandomRotate, Compose

%reload_ext tensorboard

## Hyperparameters

In [ ]:
hparams = {
    'bs': 32,
    'epochs': 100,
    'device': torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),
    'tb_logs': '/content/drive/MyDrive/Adapt',
    'tb_name': 'tb_' + str(datetime.datetime.utcnow()),
    'drive_root': '/content/drive/MyDrive/Dataset/',
    'normalize_scale': True, 
    'data_augmentation': 'flip_rotate',
    'fixed_num_of_points': 1024,
    'flip_probability': 0.5,
    'flip_axis': 1,
    'rotate_degrees': 45,
    'rotate_axis': 0,
    'model_log': '/content/drive/MyDrive/Adapt/', 
    'model_name': 'pointNet_flip_rotate_0.3_adam_OneCycleLR',
    'k': 3,
    'num_classes': 10,
    'dropout': 0.3,
    'optimizer': 'Adam',
    'lr': 1e-3,
    'wd': 1e-3,
    'momentum': 0.9,
    'scheduler': 'OneCycleLR',
    'gamma': 0.5,
    'patience': 10,
    'step_size': 20
}

## Seeds 

In [ ]:
seed = 42
# Controlling sources of randomness
torch.manual_seed(seed) # generate random numbers for all devices (both CPU and CUDA)
# Random number generators in other libraries:
np.random.seed(seed)
# CUDA convolution benchmarking:
torch.backends.cudnn.benchmark = False # ensures that CUDA selects the same algorithm each time an application is run

# Model - PointNet

In [ ]:
class TNet(nn.Module):
  def __init__(self, k=3):
    super().__init__()
    self.k = k
    
    self.Conv1 = nn.Conv1d(in_channels=k, out_channels=64, kernel_size=1)
    self.bn1 = nn.BatchNorm1d(64) 
    self.Conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=1)
    self.bn2 = nn.BatchNorm1d(128)
    self.Conv3 = nn.Conv1d(in_channels=128, out_channels=1024, kernel_size=1)
    self.bn3 = nn.BatchNorm1d(1024)

    self.FC1 = nn.Linear(in_features=1024, out_features=512)
    self.bn4 = nn.BatchNorm1d(512)
    self.FC2 = nn.Linear(in_features=512, out_features=256)
    self.bn5 = nn.BatchNorm1d(256)

    self.FC3 = nn.Linear(in_features=256, out_features=k*k)

  def forward(self, cloud_points):
    bs = cloud_points.size(0)

    x = F.relu(self.bn1(self.Conv1(cloud_points)))
    x = F.relu(self.bn2(self.Conv2(x)))
    x = F.relu(self.bn3(self.Conv3(x)))

    # size: [batch size, 1024, # of points]
    x = nn.MaxPool1d(x.size(-1))(x) # pool with kernel = # of points/batch
    # size: [batch size, 1024, 1]
    x = x.view(bs,-1) # flatten to get horizontal vector

    # size: [batch size, 1024]
    x = F.relu(self.bn4(self.FC1(x)))
    x = F.relu(self.bn5(self.FC2(x)))

    # diagonal matrices initialized, as many as batch size
    init_matrix = torch.eye(self.k, requires_grad=True).repeat(bs,1,1)
    if x.is_cuda:
      init_matrix = init_matrix.cuda() # gets updated according to f.c. output
    matrix = self.FC3(x).view(-1, self.k, self.k) + init_matrix
    return matrix

class Transform(nn.Module):
   def __init__(self, k=3):
        super().__init__()
        self.input_transform = TNet(k)
        self.feature_transform = TNet(k=64)

        self.Conv1 = nn.Conv1d(in_channels=3, out_channels=64, kernel_size=1)
        self.Conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=1)
        self.Conv3 = nn.Conv1d(in_channels=128, out_channels=1024, kernel_size=1)

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)

   def forward(self, x):
        bs = x.size(0)

        matrix3x3 = self.input_transform(x)
        x = torch.bmm(torch.transpose(x,1,2),matrix3x3).transpose(1,2)
        x = F.relu(self.bn1(self.Conv1(x)))
        
        matrix64x64 = self.feature_transform(x)
        x = torch.bmm(torch.transpose(x,1,2), matrix64x64).transpose(1,2) 
        x = F.relu(self.bn2(self.Conv2(x)))
        x = self.bn3(self.Conv3(x))

        x = nn.MaxPool1d(x.size(-1))(x)
        global_features = x.view(bs,-1)
        return global_features

class PointNetModel(nn.Module):
    def __init__(self, k=3, num_classes=16, dropout=0.3):
        super().__init__()
        self.transform = Transform(k)

        self.FC1 = nn.Linear(in_features=1024, out_features=512)
        self.bn1 = nn.BatchNorm1d(512)
        self.FC2 = nn.Linear(in_features=512, out_features=256)
        self.bn2 = nn.BatchNorm1d(256)
        self.FC3 = nn.Linear(in_features=256, out_features=num_classes)
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        global_features = self.transform(x)
        x = F.relu(self.bn1(self.FC1(global_features)))
        x = self.FC2(x)
        # apply dropout if exists
        x = self.dropout(x) if self.dropout is not None else x
        x = F.relu(self.bn2(x))
        output = self.FC3(x)
        return output, F.softmax(output,dim=1)

# Dataset
First of all, it is necessary to make the drive folder with the dataset available to this collab in order not to download it every time. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Transformations
In this project is necessary to use some transformations to either normalize the data or to perform data augmentation.

In [ ]:
def get_pre_transformation(number_points=1024):
    return SamplePoints(num=number_points)

def get_transformation(normalize_scale):
    if normalize_scale:  
      return NormalizeScale()
    else:
        return None

def get_random_flip(axis=1, p=0.5):
    return RandomFlip(axis, p)


def get_random_rotation(degrees=45, axis=1):
    return RandomRotate(degrees, axis)


def data_augmentation_flip(normalize_scale, axis=1, p=0.5):
    return Compose([get_transformation(normalize_scale), get_random_flip(axis, p)])


def data_augmentation_rotation(normalize_scale, axis=1, degrees=45):
    return Compose([get_transformation(normalize_scale), get_random_rotation(axis=axis, degrees=degrees)])


def data_augmentation_flip_rotation(normalize_scale, axis_flip=1, p=0.5, axis_rotation=1, degrees=45):
    return Compose([get_transformation(normalize_scale), get_random_flip(axis_flip, p),
                    get_random_rotation(axis=axis_rotation, degrees=degrees)])


def get_data_augmentation(dataset, transformation, normalize_scale, axis_flip=1, p=0.5, axis_rotation=1, degrees=45):
  if transformation is not None:
    if transformation.lower() == 'flip_rotation':
        dataset.transform = data_augmentation_flip_rotation(normalize_scale, axis_flip, p, axis_rotation, degrees)
    elif transformation.lower() == 'flip':
        dataset.transform = data_augmentation_flip(normalize_scale, axis=axis_flip, p=p)
    elif transformation.lower() == 'rotate':
        dataset.transform = data_augmentation_rotation(normalize_scale, axis=axis_rotation, degrees=degrees)


## Training, validation and test data

In this step, some processing of the data is going to be used to be able to feed it to the model. 

In [ ]:
def get_dataset(root, transform, pre_transform):
    train_valid_dataset = ModelNet(root=root, name="10", train=True, pre_transform=pre_transform, transform=transform)
    test_dataset = ModelNet(root=root, name="10", train=False, pre_transform=pre_transform, transform=transform)
    return train_valid_dataset, test_dataset

# Function used to split between validation and training
def get_split(index_file_root, dataset):
    index_file = open(index_file_root, 'r')
    train_index = []
    for idx in index_file:
        train_index.append(int(idx))

    return dataset[train_index]

# Function to decide which points are used in validation and which ones in training
def create_file_if_necessary(train_file, valid_file, dataset):
    if not os.path.isfile(train_file) and not os.path.isfile(valid_file):
        torch.manual_seed(0)
        # Shuffle before splitting data (random split)
        _, perm = dataset.shuffle(return_perm=True)

        # Create two files with the indices od the training and validation data
        train_idx = open(train_file, 'w+')
        valid_idx = open(valid_file, 'w+')

        # Split the tensor of indices in training and validation
        train_split, val_split = perm.split(round(len(perm) * 0.8))

        for i in range(len(train_split)):
            train_idx.writelines(str(train_split[i].item()) + "\n")
        for i in range(len(val_split)):
            valid_idx.writelines(str(val_split[i].item()) + "\n")
        
        print("New split file has been created")

        train_idx.close()
        valid_idx.close()

    elif not os.path.isfile(train_file) or not os.path.isfile(valid_file):
        raise ValueError('One file exists and the other one does not')

# Function to be called when creating dataset 
def get_train_valid_test_ModelNet(root, number_points=1024, normalize_scale=True):
    dataset_root = os.path.join(root, 'ModelNet')
    train_valid_split, test_split = get_dataset(dataset_root, transform=NormalizeScale() if normalize_scale else None,
                                                pre_transform=SamplePoints(num=number_points))

    train_split_root = os.path.join(root, 'train_split.txt')
    valid_split_root = os.path.join(root, 'val_split.txt')
    create_file_if_necessary(train_split_root, valid_split_root, train_valid_split)

    train_split = get_split(index_file_root=train_split_root, dataset=train_valid_split)
    valid_split = get_split(index_file_root=valid_split_root, dataset=train_valid_split)
    return train_split, valid_split, test_split

# Helper functions

As there are some functionalities that are used by different functions or can be used in the future, a list of helpers fucntions has been created

In [ ]:
# Initialize the tensorflow
def get_tensorboard_writer(root):
    tensorflow.io.gfile = tensorboard.compat.tensorflow_stub.io.gfile  # avoid tensorboard crash when adding embeddings
    train_log_dir = os.path.join(root, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"), 'train')
    valid_log_dir = os.path.join(root, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"), 'valid')
    train_writer = SummaryWriter(log_dir=train_log_dir)
    valid_writer = SummaryWriter(log_dir=valid_log_dir)
    return train_writer, valid_writer

# Writes information of every epoch in the tensorboard
def write_epoch_data(train_writer, valid_writer, train_loss, valid_loss, train_accuracy, valid_accuracy, epoch):
    # Write Loss and Accuracy in tensorboard:
    train_writer.add_scalar('Loss', train_loss, epoch)
    train_writer.add_scalar('Accu', train_accuracy, epoch)
    valid_writer.add_scalar('Loss', valid_loss, epoch)
    valid_writer.add_scalar('Accu', valid_accuracy, epoch)

# Funtion to save the best model so far
def update_best_model(valid_accuracy, model_state_dict, model_root, model_name):
    model_path = os.path.join(model_root, model_name + datetime.datetime.now().strftime("%Y%m%d%h"))
    torch.save(model_state_dict, model_path + '.pt')
    return valid_accuracy, model_path

# Method to visualize a cloud point
def visualize_point_cloud(point_cloud):
    points, y = point_cloud
    fig = go.Figure(data=[go.Mesh3d(x=points[1][:, 0], y=points[1][:, 1], z=points[1][:, 2], mode='markers', marker=dict(size=3, opacity=1))])
    fig.show()

# Training and testing functions

## Correct parameters
As different experiments will be performed some variables need to be dependent on hyperparameters. That is why some functions have been used due to not modify the code when performing different experiments. Adding new conditions to the if - else statement, more schedulers or optimizers can be added.

In [ ]:
def get_optimizer(optimizer_name, model_parameters, lr, wd, momentum):
    if optimizer_name.lower() == "Adam".lower():
        return torch.optim.Adam(model_parameters, lr=lr, weight_decay=wd)
    elif optimizer_name.upper() == "SGD":
        return torch.optim.SGD(model_parameters, lr=lr, momentum=momentum)
    else:
        raise ValueError('Optimizer is not correctly introduced')

def get_scheduler(scheduler_name, optimizer, lr=1e-3, gamma=0.5, patience=10, step_size=20, train_loader_len=1024,
                  num_epochs=100):
  if scheduler_name is not None:
    if scheduler_name.lower() == 'StepLR'.lower():
        return torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma), None, False
    elif scheduler_name.lower() == 'ReduceLROnPlateau'.lower():
        return torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=gamma, patience=patience), None, True
    elif scheduler_name.lower() == 'OneCycleLR'.lower():
        scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, lr, steps_per_epoch=train_loader_len,
                                                        epochs=num_epochs)
        return scheduler, scheduler, None
    else:
      raise ValueError('Incorrect scheduler')
  else:
    print('No scheduler')
    return None, None, None

## Training loop

In [ ]:
# Train One Epoch:
def train_epoch(model, train_loader, optimizer, criterion, accuracy, device, scheduler):
    # Model in train mode:
    model.train()
    # List for epoch loss:
    epoch_train_loss = []
    # Metric stored information reset:
    accuracy.reset()

    # Train epoch loop:
    for i, data in enumerate(train_loader, 1):
        # Data retrieval from each bath:
        points = to_dense_batch(data.pos, batch=data.batch)[0].to(device).float().transpose(1, 2)
        targets = data.y.to(device)

        # Forward pass:
        preds, probs = model(points)
        # Loss calculation + Backpropagation pass
        optimizer.zero_grad()
        loss = criterion(preds.to(device), targets)
        epoch_train_loss.append(loss.item())
        loss.backward()
        optimizer.step()

        # Step for OneCycle scheduler
        if scheduler is not None:
            scheduler.step()

        # Batch metrics calculation:
        accuracy.update(probs, targets)

    # Mean epoch metrics calculation:
    mean_loss = np.mean(epoch_train_loss)
    mean_accu = accuracy.compute().item()
    # Print of all metrics:
    print('Train loss: ', mean_loss, "| Acc.: " , mean_accu)
    return mean_loss, mean_accu


# Valid One Epoch:
def valid_epoch(model, valid_loader, criterion, accuracy, device):
    # Model in validation (evaluation) mode:
    model.eval()
    # List for epoch loss:
    epoch_valid_loss = []
    # Metric stored information reset:
    accuracy.reset()
    # Batch loop for validation:
    with torch.no_grad():
        for i, data in enumerate(valid_loader, 1):
            # Data retrieval from each bath:
            points = to_dense_batch(data.pos, batch=data.batch)[0].to(device).float().transpose(1, 2)
            targets = data.y.to(device)

            # Forward pass:
            preds, probs = model(points)
            # Loss calculation
            loss = criterion(preds.to(device), targets)
            epoch_valid_loss.append(loss.item())
            # Batch metrics calculation:
            accuracy.update(probs, targets)
    # Mean epoch metrics calculation:
    mean_loss = np.mean(epoch_valid_loss)
    mean_accu = accuracy.compute().item()
    # Print of all metrics:
    print('Valid loss: ', mean_loss, "| Acc.: ", mean_accu)
    return mean_loss, mean_accu


def fit(train_data, valid_data, num_classes, k=3, bs=32, num_epochs=100, lr=1e-3):
    # Data Loaders for train and validation:
    train_loader = DataLoader(train_data, batch_size=bs, shuffle=True)
    valid_loader = DataLoader(valid_data, batch_size=bs, shuffle=False)

    # Obtain correct model
    model = PointNetModel(k, num_classes, hparams['dropout']).to(hparams['device'])

    optimizer = get_optimizer(hparams['optimizer'], model.parameters(), lr, hparams['wd'], hparams['momentum'])

    # Obtain correct scheduler: train scheduler is used to determine wheteher or not there is a scheduler in the train fucntion,
    # the fit scheduler can have three values: None (no scheduler in fit function), True (needs the valid loss parameter), 
    # False(no need of extra parameter)
    scheduler, train_scheduler, fit_scheduler = get_scheduler(hparams['scheduler'], optimizer, lr, hparams['gamma'],
                                                              hparams['patience'], hparams['step_size'],
                                                              len(train_loader),
                                                              num_epochs)
    criterion = nn.CrossEntropyLoss().to(hparams['device'])

    # Metric
    accuracy = Accuracy(average='micro', compute_on_step=False).to(hparams['device'])

    # Tensorboard set up
    train_writer, valid_writer = get_tensorboard_writer(hparams['tb_logs'])

    # Minimum accuracy to save the model
    best_accuracy = 0.0
    model_root = None
    print('Start training...')
    for epoch in range(1, num_epochs + 1):
        print('Epoch: ', epoch)
        # Train and validation
        train_loss, train_accu = train_epoch(model, train_loader, optimizer, criterion, accuracy, hparams['device'],
                                             train_scheduler)
        valid_loss, valid_accu = valid_epoch(model, valid_loader, criterion, accuracy, hparams['device'])

        # Choose scheduler
        if fit_scheduler is not None:
          if fit_scheduler:
            scheduler.step(valid_loss) 
          else: 
            scheduler.step()

        write_epoch_data(train_writer, valid_writer, train_loss, valid_loss, train_accu, valid_accu, epoch)

        # Save best model:
        if best_accuracy < valid_accu:
            best_accuracy, model_root = update_best_model(valid_accu, model.state_dict(), hparams['model_log'], hparams['model_name'])

    final_state_dict_root = model_root + '.pt'
    model.load_state_dict(torch.load(final_state_dict_root))
    print("Best val accuracy: ", best_accuracy)
    return best_accuracy, final_state_dict_root

## Test inference

In [ ]:
def test(test_data, model_state_dict_root):
    test_loader = DataLoader(test_data, batch_size=1, shuffle=False)
    model = PointNetModel(hparams['k'], hparams['num_classes'], hparams['dropout']).to(hparams['device'])

    accuracy = Accuracy(average='micro', compute_on_step=False).to(hparams['device'])
    model.load_state_dict(torch.load(model_state_dict_root))
    model.eval()
    # Metric stored information reset:
    accuracy.reset()
    # Batch loop for validation:
    with torch.no_grad():
        for i, data in enumerate(test_loader, 1):
            # Data retrieval from each bath:
            points = to_dense_batch(data.pos, batch=data.batch)[0].to(hparams['device']).float().transpose(1, 2)
            targets = data.y.to(hparams['device'])

            # Forward pass:
            preds, probs = model(points)

            # Batch metrics calculation:
            accuracy.update(probs, targets)

    mean_accu = accuracy.compute().item()
    # Print of all metrics:
    print("Test Acc.: ", mean_accu)
    return mean_accu

# Experiments
More info in the README.

In [ ]:
def execute_experiment():
  train_dataset, valid_dataset, test_dataset = get_train_valid_test_ModelNet(hparams['drive_root'], hparams['fixed_num_of_points'], hparams['normalize_scale'])
  get_data_augmentation(train_dataset, hparams['data_augmentation'], hparams['normalize_scale'], hparams['flip_axis'], hparams['flip_probability'],
                        hparams['rotate_axis'], hparams['rotate_degrees'])
  best_acc, state_dict_root = fit(train_dataset, valid_dataset, hparams['num_classes'], hparams['k'], hparams['bs'],
                                  hparams['epochs'], hparams['lr'])

  test_inference = test(test_dataset, state_dict_root)

def get_model_name(model, normalize_scale, data_augmentation, dropout, optimizer, scheduler):
  model_name = model
  model_name += '_normalized' if normalize_scale else '_notNormalized'
  model_name += "_" + data_augmentation if data_augmentation is not None else '_noDataAugmentation'
  model_name += "_" + str(dropout) if dropout is not None else '_noDropout'
  model_name += "_" + optimizer
  model_name += "_" + scheduler if scheduler is not None else "_noScheduler"
  return model_name

In [ ]:
# Experiment 1
hparams['optimizer'] = 'Adam'
hparams['normalize_scale'] = False
hparams['dropout'] = None
hparams['scheduler'] = None
hparams['data_augmentation'] = None
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()


No scheduler
Start training...
Epoch:  1
Train loss:  2.178066202402115 | Acc.:  0.2148449718952179
Valid loss:  2.2247424507141114 | Acc.:  0.20426064729690552
Epoch:  2
Train loss:  1.9852601718902587 | Acc.:  0.30598184466362
Valid loss:  107.63025043487549 | Acc.:  0.20551379024982452
Epoch:  3
Train loss:  1.8341474997997285 | Acc.:  0.35295960307121277
Valid loss:  2.342557487487793 | Acc.:  0.2017543911933899
Epoch:  4
Train loss:  1.7432118153572083 | Acc.:  0.37143751978874207
Valid loss:  2.559122905731201 | Acc.:  0.2030075192451477
Epoch:  5
Train loss:  1.7071514761447906 | Acc.:  0.38083305954933167
Valid loss:  3.78560338973999 | Acc.:  0.2030075192451477
Epoch:  6
Train loss:  1.6388303792476655 | Acc.:  0.39116817712783813
Valid loss:  2.4020598220825193 | Acc.:  0.20551379024982452
Epoch:  7
Train loss:  1.6749877607822419 | Acc.:  0.3971186876296997
Valid loss:  4.193084621429444 | Acc.:  0.25313282012939453
Epoch:  8
Train loss:  1.5749424123764038 | Acc.:  0.420920

In [ ]:
# Experiment 2
hparams['optimizer'] = 'Adam'
hparams['normalize_scale'] = True
hparams['dropout'] = None
hparams['scheduler'] = None
hparams['data_augmentation'] = None
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

No scheduler
Start training...
Epoch:  1
Train loss:  0.9015751945972442 | Acc.:  0.7150015830993652
Valid loss:  0.6778614819049835 | Acc.:  0.7531328201293945
Epoch:  2
Train loss:  0.5359513311088085 | Acc.:  0.8264954686164856
Valid loss:  0.46273725628852846 | Acc.:  0.8621553778648376
Epoch:  3
Train loss:  0.40837384983897207 | Acc.:  0.8668963313102722
Valid loss:  1.1395153307914734 | Acc.:  0.7243107557296753
Epoch:  4
Train loss:  0.3806937648355961 | Acc.:  0.8712809085845947
Valid loss:  0.39580806016922 | Acc.:  0.878446102142334
Epoch:  5
Train loss:  0.3438349691778421 | Acc.:  0.8916379809379578
Valid loss:  0.3682044112682343 | Acc.:  0.8709273338317871
Epoch:  6
Train loss:  0.3447312714904547 | Acc.:  0.8872533440589905
Valid loss:  0.40179804623126986 | Acc.:  0.8822054862976074
Epoch:  7
Train loss:  0.280577926710248 | Acc.:  0.9076104164123535
Valid loss:  0.5167097353935242 | Acc.:  0.8095238208770752
Epoch:  8
Train loss:  0.3002659369260073 | Acc.:  0.9041653

In [ ]:
# Experiment 3
hparams['optimizer'] = 'Adam'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = None
hparams['data_augmentation'] = None
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

No scheduler
Start training...
Epoch:  1
Train loss:  0.8640682485699653 | Acc.:  0.7262762188911438
Valid loss:  0.7534669530391693 | Acc.:  0.7669172883033752
Epoch:  2
Train loss:  0.5604299156367779 | Acc.:  0.8186658024787903
Valid loss:  0.7167724394798278 | Acc.:  0.7769423723220825
Epoch:  3
Train loss:  0.41740616992115975 | Acc.:  0.8590667247772217
Valid loss:  0.48880519926548005 | Acc.:  0.8458646535873413
Epoch:  4
Train loss:  0.3637230710685253 | Acc.:  0.8778578042984009
Valid loss:  0.4334406566619873 | Acc.:  0.8583959937095642
Epoch:  5
Train loss:  0.33993335872888564 | Acc.:  0.8878797292709351
Valid loss:  0.5366975569725037 | Acc.:  0.8195488452911377
Epoch:  6
Train loss:  0.37309373304247856 | Acc.:  0.8737863898277283
Valid loss:  0.7175619339942932 | Acc.:  0.7706766724586487
Epoch:  7
Train loss:  0.32673811823129656 | Acc.:  0.8878797292709351
Valid loss:  0.5042479300498962 | Acc.:  0.8320801854133606
Epoch:  8
Train loss:  0.29326229602098464 | Acc.:  0.

In [ ]:
# Experiment 4
hparams['optimizer'] = 'Adam'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = None
hparams['data_augmentation'] = 'flip'
hparams['epochs'] = 100 
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

No scheduler
Start training...
Epoch:  1
Train loss:  1.0747228974103928 | Acc.:  0.6592546105384827
Valid loss:  0.8828669393062591 | Acc.:  0.7556390762329102
Epoch:  2
Train loss:  0.5594667685031891 | Acc.:  0.8142812252044678
Valid loss:  0.7137234950065613 | Acc.:  0.7832080125808716
Epoch:  3
Train loss:  0.45670479983091355 | Acc.:  0.851550281047821
Valid loss:  0.8106449115276336 | Acc.:  0.719298243522644
Epoch:  4
Train loss:  0.39016166716814044 | Acc.:  0.8687754273414612
Valid loss:  0.6590721994638443 | Acc.:  0.7907268404960632
Epoch:  5
Train loss:  0.3545643599331379 | Acc.:  0.8816160559654236
Valid loss:  0.4435738134384155 | Acc.:  0.853383481502533
Epoch:  6
Train loss:  0.3028637260943651 | Acc.:  0.8985280394554138
Valid loss:  0.2679069027304649 | Acc.:  0.9223057627677917
Epoch:  7
Train loss:  0.3071896656602621 | Acc.:  0.8969621062278748
Valid loss:  0.33373925238847735 | Acc.:  0.8922305703163147
Epoch:  8
Train loss:  0.30482972010970116 | Acc.:  0.89946

In [ ]:
# Experiment 5
hparams['optimizer'] = 'Adam'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = None
hparams['data_augmentation'] = 'rotate'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

No scheduler
Start training...
Epoch:  1
Train loss:  1.1976808667182923 | Acc.:  0.6216723918914795
Valid loss:  1.0485394620895385 | Acc.:  0.6867167949676514
Epoch:  2
Train loss:  0.7515445294976234 | Acc.:  0.7626057267189026
Valid loss:  1.01006254196167 | Acc.:  0.6654135584831238
Epoch:  3
Train loss:  0.6221455751359463 | Acc.:  0.7926714420318604
Valid loss:  0.6693071258068085 | Acc.:  0.7819548845291138
Epoch:  4
Train loss:  0.5802152484655381 | Acc.:  0.8111494183540344
Valid loss:  0.5505079305171967 | Acc.:  0.7907268404960632
Epoch:  5
Train loss:  0.5063884529471397 | Acc.:  0.8362041711807251
Valid loss:  0.5282450884580612 | Acc.:  0.8345864415168762
Epoch:  6
Train loss:  0.43402639478445054 | Acc.:  0.8502975106239319
Valid loss:  0.9203149724006653 | Acc.:  0.6654135584831238
Epoch:  7
Train loss:  0.42923244208097455 | Acc.:  0.856874406337738
Valid loss:  0.43029202938079836 | Acc.:  0.8333333134651184
Epoch:  8
Train loss:  0.4225604572892189 | Acc.:  0.859379

In [ ]:
# Experiment 6
hparams['optimizer'] = 'Adam'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = None
hparams['data_augmentation'] = 'flip_rotation'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

No scheduler
Start training...
Epoch:  1
Train loss:  1.280290659070015 | Acc.:  0.5850297808647156
Valid loss:  0.9622425866127015 | Acc.:  0.7092731595039368
Epoch:  2
Train loss:  0.7652142325043678 | Acc.:  0.7579079270362854
Valid loss:  0.705112532377243 | Acc.:  0.7832080125808716
Epoch:  3
Train loss:  0.6379380491375923 | Acc.:  0.796742856502533
Valid loss:  0.7804261672496796 | Acc.:  0.7894737124443054
Epoch:  4
Train loss:  0.5995183178782463 | Acc.:  0.8083307147026062
Valid loss:  0.5182466518878937 | Acc.:  0.8270676732063293
Epoch:  5
Train loss:  0.5112033309042454 | Acc.:  0.8236767649650574
Valid loss:  0.6835712718963624 | Acc.:  0.7769423723220825
Epoch:  6
Train loss:  0.4982883954048157 | Acc.:  0.8421547412872314
Valid loss:  1.153480839729309 | Acc.:  0.6203007698059082
Epoch:  7
Train loss:  0.4849218209087849 | Acc.:  0.8459129333496094
Valid loss:  0.9040483903884887 | Acc.:  0.6741854548454285
Epoch:  8
Train loss:  0.45848562151193617 | Acc.:  0.847478866

In [ ]:
# Experiment 7
hparams['optimizer'] = 'Adam'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = 'StepLR'
hparams['data_augmentation'] = 'flip'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

Start training...
Epoch:  1


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning:

Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)



Train loss:  0.9976083487272263 | Acc.:  0.684622585773468
Valid loss:  0.9071384620666504 | Acc.:  0.7180451154708862
Epoch:  2
Train loss:  0.582380641400814 | Acc.:  0.8139680624008179
Valid loss:  0.722077077627182 | Acc.:  0.7744361162185669
Epoch:  3
Train loss:  0.4362652578949928 | Acc.:  0.8603194355964661
Valid loss:  0.823348650932312 | Acc.:  0.7042606472969055
Epoch:  4
Train loss:  0.400647075176239 | Acc.:  0.8687754273414612
Valid loss:  0.7061478137969971 | Acc.:  0.7518796920776367
Epoch:  5
Train loss:  0.40052102170884607 | Acc.:  0.8684622645378113
Valid loss:  0.3627765882015228 | Acc.:  0.8796992301940918
Epoch:  6
Train loss:  0.32584119483828544 | Acc.:  0.8947697877883911
Valid loss:  0.43957046687602996 | Acc.:  0.8646616339683533
Epoch:  7
Train loss:  0.31776060357689856 | Acc.:  0.9000939726829529
Valid loss:  0.46226177871227264 | Acc.:  0.8408521413803101
Epoch:  8
Train loss:  0.30264324449002744 | Acc.:  0.9032257795333862
Valid loss:  0.53305461525917

In [ ]:
# Experiment 8
hparams['optimizer'] = 'Adam'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = 'StepLR'
hparams['data_augmentation'] = 'flip_rotation'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

Start training...
Epoch:  1
Train loss:  1.313915086388588 | Acc.:  0.5684309601783752
Valid loss:  1.1911603593826294 | Acc.:  0.6441102623939514
Epoch:  2
Train loss:  0.7859119489789009 | Acc.:  0.7488255500793457
Valid loss:  0.9054139316082 | Acc.:  0.7318295836448669
Epoch:  3
Train loss:  0.6460938262939453 | Acc.:  0.7964296936988831
Valid loss:  0.9257501137256622 | Acc.:  0.6616541147232056
Epoch:  4
Train loss:  0.5852105025947094 | Acc.:  0.8111494183540344
Valid loss:  0.6115162354707718 | Acc.:  0.7944862246513367
Epoch:  5
Train loss:  0.5159673120081425 | Acc.:  0.8343250751495361
Valid loss:  0.505831562280655 | Acc.:  0.8421052694320679
Epoch:  6
Train loss:  0.4777345606684685 | Acc.:  0.8528029918670654
Valid loss:  0.38991354405879974 | Acc.:  0.8746867179870605
Epoch:  7
Train loss:  0.4455972224473953 | Acc.:  0.8528029918670654
Valid loss:  0.3906331050395966 | Acc.:  0.8809523582458496
Epoch:  8
Train loss:  0.41364057585597036 | Acc.:  0.8643908500671387
Valid

In [ ]:
# Experiment 9
hparams['optimizer'] = 'Adam'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = 'OneCycleLR'
hparams['data_augmentation'] = 'flip'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

Start training...
Epoch:  1
Train loss:  1.663453003168106 | Acc.:  0.49953022599220276
Valid loss:  1.4117224502563477 | Acc.:  0.6666666865348816
Epoch:  2
Train loss:  0.9228683894872666 | Acc.:  0.7644848227500916
Valid loss:  0.7503936505317688 | Acc.:  0.8007518649101257
Epoch:  3
Train loss:  0.6255081361532211 | Acc.:  0.8315064311027527
Valid loss:  0.6110695636272431 | Acc.:  0.835839569568634
Epoch:  4
Train loss:  0.4857858416438103 | Acc.:  0.8665831685066223
Valid loss:  0.6057030057907105 | Acc.:  0.8057644367218018
Epoch:  5
Train loss:  0.4102075293660164 | Acc.:  0.8841215372085571
Valid loss:  0.5500545120239257 | Acc.:  0.8483709096908569
Epoch:  6
Train loss:  0.3779329486191273 | Acc.:  0.8941434621810913
Valid loss:  0.4386025953292847 | Acc.:  0.8759398460388184
Epoch:  7
Train loss:  0.36689957104623316 | Acc.:  0.8969621062278748
Valid loss:  0.5339282947778702 | Acc.:  0.8320801854133606
Epoch:  8
Train loss:  0.32080298982560634 | Acc.:  0.9013466835021973
V

In [ ]:
# Experiment 10
hparams['optimizer'] = 'Adam'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = 'OneCycleLR'
hparams['data_augmentation'] = 'flip_rotation'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

Start training...
Epoch:  1
Train loss:  1.815625422000885 | Acc.:  0.40839335322380066
Valid loss:  1.3710240364074706 | Acc.:  0.5651628971099854
Epoch:  2
Train loss:  1.0981081861257553 | Acc.:  0.6911994814872742
Valid loss:  0.9047474217414856 | Acc.:  0.7556390762329102
Epoch:  3
Train loss:  0.7807281777262688 | Acc.:  0.7716880440711975
Valid loss:  0.6936631190776825 | Acc.:  0.8020049929618835
Epoch:  4
Train loss:  0.6531098788976669 | Acc.:  0.8114625811576843
Valid loss:  0.7837804770469665 | Acc.:  0.7556390762329102
Epoch:  5
Train loss:  0.585497260093689 | Acc.:  0.8252426981925964
Valid loss:  0.560333663225174 | Acc.:  0.8571428656578064
Epoch:  6
Train loss:  0.5218447068333626 | Acc.:  0.844660222530365
Valid loss:  0.4493879735469818 | Acc.:  0.8721804618835449
Epoch:  7
Train loss:  0.4701493553817272 | Acc.:  0.8562480211257935
Valid loss:  0.5707712459564209 | Acc.:  0.8208020329475403
Epoch:  8
Train loss:  0.4071396581828594 | Acc.:  0.8719072937965393
Valid

In [ ]:
# Experiment 11
hparams['optimizer'] = 'SGD'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = None
hparams['data_augmentation'] = 'flip_rotation'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

No scheduler
Start training...
Epoch:  1
Train loss:  1.6091786390542984 | Acc.:  0.48199185729026794
Valid loss:  1.2806156826019288 | Acc.:  0.6190476417541504
Epoch:  2
Train loss:  0.9787459141016006 | Acc.:  0.7059192061424255
Valid loss:  0.9265585494041443 | Acc.:  0.719298243522644
Epoch:  3
Train loss:  0.7418433156609535 | Acc.:  0.7688694000244141
Valid loss:  0.5893815815448761 | Acc.:  0.8233082890510559
Epoch:  4
Train loss:  0.6513323737680912 | Acc.:  0.7926714420318604
Valid loss:  0.538126186132431 | Acc.:  0.8471177816390991
Epoch:  5
Train loss:  0.5585473018884659 | Acc.:  0.8264954686164856
Valid loss:  0.422204926609993 | Acc.:  0.8759398460388184
Epoch:  6
Train loss:  0.48178444102406504 | Acc.:  0.8490447998046875
Valid loss:  0.5285448920726776 | Acc.:  0.8208020329475403
Epoch:  7
Train loss:  0.4348058319836855 | Acc.:  0.8625117540359497
Valid loss:  0.6312997555732727 | Acc.:  0.8182957172393799
Epoch:  8
Train loss:  0.41632733821868895 | Acc.:  0.872846

In [ ]:
# Experiment 12
hparams['optimizer'] = 'SGD'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = 'StepLR'
hparams['data_augmentation'] = 'flip_rotation'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

Start training...
Epoch:  1


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning:

Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)



Train loss:  1.6861221444606782 | Acc.:  0.4704039990901947
Valid loss:  1.1869862747192383 | Acc.:  0.6340852379798889
Epoch:  2
Train loss:  0.9067869579792023 | Acc.:  0.7278421521186829
Valid loss:  0.7568992364406586 | Acc.:  0.7832080125808716
Epoch:  3
Train loss:  0.6553966104984283 | Acc.:  0.7961165308952332
Valid loss:  0.5679258692264557 | Acc.:  0.8220551609992981
Epoch:  4
Train loss:  0.5622555321455002 | Acc.:  0.8177262544631958
Valid loss:  0.5003283327817917 | Acc.:  0.8508771657943726
Epoch:  5
Train loss:  0.48110232159495353 | Acc.:  0.8496711850166321
Valid loss:  0.4147045290470123 | Acc.:  0.8746867179870605
Epoch:  6
Train loss:  0.42945205882191656 | Acc.:  0.8684622645378113
Valid loss:  0.34826371490955355 | Acc.:  0.897243082523346
Epoch:  7
Train loss:  0.34439962908625604 | Acc.:  0.8925775289535522
Valid loss:  0.41243417978286745 | Acc.:  0.8634085059165955
Epoch:  8
Train loss:  0.34427591480314734 | Acc.:  0.8922643065452576
Valid loss:  0.4365559542

In [ ]:
# Experiment 13
hparams['optimizer'] = 'SGD'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = 'OneCycleLR'
hparams['data_augmentation'] = 'flip_rotation'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

Start training...
Epoch:  1
Train loss:  2.171976714134216 | Acc.:  0.25806450843811035
Valid loss:  1.9398072814941407 | Acc.:  0.4536340832710266
Epoch:  2
Train loss:  1.7512228274345398 | Acc.:  0.480112761259079
Valid loss:  1.6025058889389039 | Acc.:  0.5275689363479614
Epoch:  3
Train loss:  1.444397885799408 | Acc.:  0.5759474039077759
Valid loss:  1.2459260177612306 | Acc.:  0.6340852379798889
Epoch:  4
Train loss:  1.2180867993831634 | Acc.:  0.6451612710952759
Valid loss:  1.077134335041046 | Acc.:  0.6829574108123779
Epoch:  5
Train loss:  1.0412954902648925 | Acc.:  0.7084246873855591
Valid loss:  0.8717819094657898 | Acc.:  0.7581453919410706
Epoch:  6
Train loss:  0.8901121735572814 | Acc.:  0.7491387128829956
Valid loss:  0.7193553853034973 | Acc.:  0.8082706928253174
Epoch:  7
Train loss:  0.7705243548750877 | Acc.:  0.7813968062400818
Valid loss:  0.7049560785293579 | Acc.:  0.8020049929618835
Epoch:  8
Train loss:  0.6851157999038696 | Acc.:  0.8030065894126892
Valid

In [ ]:
# Experiment 14
hparams['optimizer'] = 'SGD'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = None
hparams['data_augmentation'] = 'flip'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

No scheduler
Start training...
Epoch:  1
Train loss:  1.3729994505643845 | Acc.:  0.5822110772132874
Valid loss:  0.9430013728141785 | Acc.:  0.7142857313156128
Epoch:  2
Train loss:  0.6722740578651428 | Acc.:  0.8089570999145508
Valid loss:  0.6086430478096009 | Acc.:  0.810776948928833
Epoch:  3
Train loss:  0.4924004362523556 | Acc.:  0.856874406337738
Valid loss:  0.44150442004203794 | Acc.:  0.8796992301940918
Epoch:  4
Train loss:  0.3976084142923355 | Acc.:  0.8860006332397461
Valid loss:  0.42565572023391723 | Acc.:  0.8684210777282715
Epoch:  5
Train loss:  0.3453407160937786 | Acc.:  0.8881929516792297
Valid loss:  0.4843659782409668 | Acc.:  0.8333333134651184
Epoch:  6
Train loss:  0.33885488979518413 | Acc.:  0.8953961730003357
Valid loss:  0.291295665204525 | Acc.:  0.9010025262832642
Epoch:  7
Train loss:  0.28757316917181014 | Acc.:  0.913874089717865
Valid loss:  0.3321090295910835 | Acc.:  0.8922305703163147
Epoch:  8
Train loss:  0.2642091703414917 | Acc.:  0.910115

In [ ]:
# Experiment 15
hparams['optimizer'] = 'SGD'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = 'StepLR'
hparams['data_augmentation'] = 'flip'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

Start training...
Epoch:  1
Train loss:  1.4437350863218308 | Acc.:  0.5502662062644958
Valid loss:  0.920036723613739 | Acc.:  0.756892204284668
Epoch:  2
Train loss:  0.6883337819576263 | Acc.:  0.7979956269264221
Valid loss:  0.6495165991783142 | Acc.:  0.7994987368583679
Epoch:  3
Train loss:  0.5291914187371731 | Acc.:  0.8358910083770752
Valid loss:  0.5725218856334686 | Acc.:  0.8245614171028137
Epoch:  4
Train loss:  0.4582995367050171 | Acc.:  0.8609458208084106
Valid loss:  0.37364447355270386 | Acc.:  0.8959899544715881
Epoch:  5
Train loss:  0.37245652921497824 | Acc.:  0.8944566249847412
Valid loss:  0.3783905231952667 | Acc.:  0.8947368264198303
Epoch:  6
Train loss:  0.34040675960481165 | Acc.:  0.8975884914398193
Valid loss:  0.4568136021494865 | Acc.:  0.8796992301940918
Epoch:  7
Train loss:  0.2967221334576607 | Acc.:  0.906670868396759
Valid loss:  0.33295207649469377 | Acc.:  0.8809523582458496
Epoch:  8
Train loss:  0.2609803029149771 | Acc.:  0.9141873121261597
V

In [ ]:
# Experiment 16
hparams['optimizer'] = 'SGD'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = 'OneCycleLR'
hparams['data_augmentation'] = 'flip'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

Start training...
Epoch:  1
Train loss:  2.1216506719589234 | Acc.:  0.27685561776161194
Valid loss:  1.7892072486877442 | Acc.:  0.523809552192688
Epoch:  2
Train loss:  1.5708397245407104 | Acc.:  0.5725023746490479
Valid loss:  1.3230085563659668 | Acc.:  0.6190476417541504
Epoch:  3
Train loss:  1.2189967453479766 | Acc.:  0.6673974394798279
Valid loss:  1.0129253101348876 | Acc.:  0.7368420958518982
Epoch:  4
Train loss:  0.9856185436248779 | Acc.:  0.755089282989502
Valid loss:  0.8046073532104492 | Acc.:  0.8258145451545715
Epoch:  5
Train loss:  0.7705601179599761 | Acc.:  0.8030065894126892
Valid loss:  0.6495025527477264 | Acc.:  0.8270676732063293
Epoch:  6
Train loss:  0.6462751242518425 | Acc.:  0.8296273350715637
Valid loss:  0.6099048256874084 | Acc.:  0.8583959937095642
Epoch:  7
Train loss:  0.5522373604774475 | Acc.:  0.8490447998046875
Valid loss:  0.5030016267299652 | Acc.:  0.8634085059165955
Epoch:  8
Train loss:  0.4821507214009762 | Acc.:  0.8694018125534058
Val